In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [29]:
def getData():
    r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
    ratings = pd.read_csv('u.data', sep='\t', names=r_cols,
                          encoding='latin-1')
    m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
    movies = pd.read_csv('u.item', sep='|', names=m_cols, usecols=range(5),
                         encoding='latin-1')
    movie_ratings = pd.merge(movies, ratings)
    temp = movie_ratings[['movie_id', 'user_id', 'rating']].copy()
    temp = temp.pivot_table(columns='movie_id', index='user_id', values='rating').copy()
    temp.index = ['User_'+str(int(i)) for i in temp.index]
    temp.columns = ['Filme_'+str(int(i)) for i in temp.columns]
    qtd_cols = 80
    R = temp.iloc[:, :qtd_cols]
    l=[]
    for i in range(1, R.shape[0]+1):
        if R.iloc[i-1, ].isnull().sum() >= (qtd_cols - 10):
            l.append(i)
    R = R.drop(["User_"+str(r) for r in l])
    R.index = ['User_'+str(int(i)) for i in range(R.shape[0])]
    return R

In [30]:
import time

class MatrixFactorization():
    
    def __init__(self, dataframe, K, steps, alpha, beta):
        self.df = dataframe
        self.K = K
        self.steps = steps
        self.alpha = alpha
        self.beta = beta
        
    def fit(self):
        t0 = time.time()
        
        R = self.df.values
        N, M = R.shape
        
        #inicio aleatorio
        P = np.random.rand(N,self.K)
        Q = np.random.rand(self.K,M)
        
        lista_erro_step = []
        
        #loop
        for step in range(self.steps):
            
            mse_total_step = 0
            #varrendo todas as entradas da matriz R
            for i in range(len(R)):
                for j in range(len(R[i])):
                    #validando se o valor associado está preenchido
                    if R[i][j] > 0:

                        #calculando o erro:
                        eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                        mse_total_step += (eij)**2
                        #alterando os valores
                        for k in range(self.K):
                            P[i][k] = P[i][k] + self.alpha * ( 2 * eij * Q[k][j] - self.beta * P[i][k])
                            Q[k][j] = Q[k][j] + self.alpha * ( 2 * eij * P[i][k] - self.beta * Q[k][j])
                            
            lista_erro_step.append(mse_total_step)
            
        self.P = P
        self.Q = Q
        self.lista_erro_step = lista_erro_step
        t1 = time.time()
        print("Fatoração concluída. Tempo aproximado:", int((t1-t0)/60)+1, 'minuto(s).')
        
    def predict(self):
        return self.P.dot(self.Q)
    
    def print_MSE_steps(self):
        plt.figure(figsize=[15,6])
        plt.title("Custo total por Step", fontsize = 16, fontweight = 'bold')
        plt.xlabel("Step", fontsize = 14, fontweight = 'bold')
        plt.ylabel("Erro", fontsize = 14, fontweight = 'bold')
        plt.plot(range(1, 1+self.steps), self.lista_erro_step, c = 'blue', lw = 2)
        plt.grid()
        plt.show()

In [31]:
def getDF():
    dic__ = {"User_1":[np.nan, np.nan, np.nan, 1, 7, 2, 3, 8],
         "User_2":[9,10,2,2,6,5,3,8],
         "User_3":[4, 7, 9, 6,6,10,10,2],
         "User_4":[np.nan, 7, 9, 5, 5, 10, 9, 1],
         "User_5":[7.0,6.0,3.0,8.0,3,4.0,3.0, 2],
         "User_6":[np.nan, np.nan, 9, 9,6,8,9,np.nan],
         "User_7":[3,5,4,4,3,3,9,np.nan],
         "User_8":[10,10,10,10,2,2,2,2],
         "User_9":[9,9,np.nan,8,3,3,1,np.nan],
         "User_10":[9,8,10,9,3,4,2,1],
         "User_11":[4,4,3,3,9,9,8,10],
         "User_12":[2,2,4,1,8,10,10,9],
         "User_13":[1,4,1,3,7,10,7,8],
         "User_14":[3,3,2,1,1,10, np.nan,10],
         "User_15":[9,9,8,10,4,2,np.nan,1]
        }
    df = pd.DataFrame(dic__).T
    df.columns = ['Filme_'+str(int(i+1)) for i in range(8)]
    return df

In [32]:
df = getDF()
df

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0
User_3,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0
User_4,NaN,7.0,9.0,5.0,5.0,10.0,9.0,1.0
User_5,7.0,6.0,3.0,8.0,3.0,4.0,3.0,2.0
User_6,NaN,NaN,9.0,9.0,6.0,8.0,9.0,NaN
User_7,3.0,5.0,4.0,4.0,3.0,3.0,9.0,NaN
User_8,10.0,10.0,10.0,10.0,2.0,2.0,2.0,2.0
User_9,9.0,9.0,NaN,8.0,3.0,3.0,1.0,NaN
User_10,9.0,8.0,10.0,9.0,3.0,4.0,2.0,1.0


In [33]:
ratings = df.fillna(0).values
ratings

array([[ 0.,  0.,  0.,  1.,  7.,  2.,  3.,  8.],
       [ 9., 10.,  2.,  2.,  6.,  5.,  3.,  8.],
       [ 4.,  7.,  9.,  6.,  6., 10., 10.,  2.],
       [ 0.,  7.,  9.,  5.,  5., 10.,  9.,  1.],
       [ 7.,  6.,  3.,  8.,  3.,  4.,  3.,  2.],
       [ 0.,  0.,  9.,  9.,  6.,  8.,  9.,  0.],
       [ 3.,  5.,  4.,  4.,  3.,  3.,  9.,  0.],
       [10., 10., 10., 10.,  2.,  2.,  2.,  2.],
       [ 9.,  9.,  0.,  8.,  3.,  3.,  1.,  0.],
       [ 9.,  8., 10.,  9.,  3.,  4.,  2.,  1.],
       [ 4.,  4.,  3.,  3.,  9.,  9.,  8., 10.],
       [ 2.,  2.,  4.,  1.,  8., 10., 10.,  9.],
       [ 1.,  4.,  1.,  3.,  7., 10.,  7.,  8.],
       [ 3.,  3.,  2.,  1.,  1., 10.,  0., 10.],
       [ 9.,  9.,  8., 10.,  4.,  2.,  0.,  1.]])

In [34]:
def train_test_split(ratings, qtd):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=qtd, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    return train, test

In [35]:
train, test = train_test_split(ratings, qtd = 2)

In [36]:
ratings

array([[ 0.,  0.,  0.,  1.,  7.,  2.,  3.,  8.],
       [ 9., 10.,  2.,  2.,  6.,  5.,  3.,  8.],
       [ 4.,  7.,  9.,  6.,  6., 10., 10.,  2.],
       [ 0.,  7.,  9.,  5.,  5., 10.,  9.,  1.],
       [ 7.,  6.,  3.,  8.,  3.,  4.,  3.,  2.],
       [ 0.,  0.,  9.,  9.,  6.,  8.,  9.,  0.],
       [ 3.,  5.,  4.,  4.,  3.,  3.,  9.,  0.],
       [10., 10., 10., 10.,  2.,  2.,  2.,  2.],
       [ 9.,  9.,  0.,  8.,  3.,  3.,  1.,  0.],
       [ 9.,  8., 10.,  9.,  3.,  4.,  2.,  1.],
       [ 4.,  4.,  3.,  3.,  9.,  9.,  8., 10.],
       [ 2.,  2.,  4.,  1.,  8., 10., 10.,  9.],
       [ 1.,  4.,  1.,  3.,  7., 10.,  7.,  8.],
       [ 3.,  3.,  2.,  1.,  1., 10.,  0., 10.],
       [ 9.,  9.,  8., 10.,  4.,  2.,  0.,  1.]])

In [37]:
train

array([[ 0.,  0.,  0.,  0.,  7.,  2.,  3.,  0.],
       [ 9.,  0.,  2.,  0.,  6.,  5.,  3.,  8.],
       [ 0.,  7.,  9.,  6.,  6., 10.,  0.,  2.],
       [ 0.,  7.,  9.,  5.,  5.,  0.,  0.,  1.],
       [ 7.,  0.,  3.,  8.,  3.,  4.,  0.,  2.],
       [ 0.,  0.,  0.,  0.,  6.,  8.,  9.,  0.],
       [ 0.,  5.,  4.,  4.,  3.,  3.,  0.,  0.],
       [10.,  0., 10.,  0.,  2.,  2.,  2.,  2.],
       [ 9.,  9.,  0.,  0.,  3.,  0.,  1.,  0.],
       [ 9.,  8., 10.,  0.,  3.,  4.,  2.,  0.],
       [ 4.,  0.,  3.,  3.,  9.,  9.,  0., 10.],
       [ 0.,  2.,  4.,  1.,  8., 10., 10.,  0.],
       [ 1.,  4.,  0.,  0.,  7., 10.,  7.,  8.],
       [ 3.,  3.,  2.,  1.,  1.,  0.,  0.,  0.],
       [ 9.,  9.,  8., 10.,  0.,  0.,  0.,  1.]])

In [38]:
test

array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  8.],
       [ 0., 10.,  0.,  2.,  0.,  0.,  0.,  0.],
       [ 4.,  0.,  0.,  0.,  0.,  0., 10.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 10.,  9.,  0.],
       [ 0.,  6.,  0.,  0.,  0.,  0.,  3.,  0.],
       [ 0.,  0.,  9.,  9.,  0.,  0.,  0.,  0.],
       [ 3.,  0.,  0.,  0.,  0.,  0.,  9.,  0.],
       [ 0., 10.,  0., 10.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  8.,  0.,  3.,  0.,  0.],
       [ 0.,  0.,  0.,  9.,  0.,  0.,  0.,  1.],
       [ 0.,  4.,  0.,  0.,  0.,  0.,  8.,  0.],
       [ 2.,  0.,  0.,  0.,  0.,  0.,  0.,  9.],
       [ 0.,  0.,  1.,  3.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 10.,  0., 10.],
       [ 0.,  0.,  0.,  0.,  4.,  2.,  0.,  0.]])

In [39]:
train, val = train_test_split(train, qtd = 2)

In [40]:
train

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  3.,  0.],
       [ 9.,  0.,  2.,  0.,  0.,  5.,  0.,  8.],
       [ 0.,  7.,  9.,  6.,  6.,  0.,  0.,  0.],
       [ 0.,  7.,  9.,  0.,  5.,  0.,  0.,  0.],
       [ 7.,  0.,  3.,  0.,  3.,  4.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  6.,  0.,  0.,  0.],
       [ 0.,  0.,  4.,  0.,  3.,  3.,  0.,  0.],
       [10.,  0., 10.,  0.,  2.,  0.,  0.,  2.],
       [ 0.,  9.,  0.,  0.,  3.,  0.,  0.,  0.],
       [ 9.,  8.,  0.,  0.,  3.,  0.,  2.,  0.],
       [ 4.,  0.,  3.,  3.,  0.,  9.,  0.,  0.],
       [ 0.,  2.,  4.,  0.,  8., 10.,  0.,  0.],
       [ 1.,  4.,  0.,  0.,  0., 10.,  7.,  0.],
       [ 0.,  3.,  2.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  9.,  8.,  0.,  0.,  0.,  0.,  1.]])

In [41]:
val

array([[ 0.,  0.,  0.,  0.,  7.,  2.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  6.,  0.,  3.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 10.,  0.,  2.],
       [ 0.,  0.,  0.,  5.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  8.,  0.,  0.,  0.,  2.],
       [ 0.,  0.,  0.,  0.,  0.,  8.,  9.,  0.],
       [ 0.,  5.,  0.,  4.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  2.,  2.,  0.],
       [ 9.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0., 10.,  0.,  0.,  4.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  9.,  0.,  0., 10.],
       [ 0.,  0.,  0.,  1.,  0.,  0., 10.,  0.],
       [ 0.,  0.,  0.,  0.,  7.,  0.,  0.,  8.],
       [ 3.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 9.,  0.,  0., 10.,  0.,  0.,  0.,  0.]])

In [42]:
test

array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  8.],
       [ 0., 10.,  0.,  2.,  0.,  0.,  0.,  0.],
       [ 4.,  0.,  0.,  0.,  0.,  0., 10.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 10.,  9.,  0.],
       [ 0.,  6.,  0.,  0.,  0.,  0.,  3.,  0.],
       [ 0.,  0.,  9.,  9.,  0.,  0.,  0.,  0.],
       [ 3.,  0.,  0.,  0.,  0.,  0.,  9.,  0.],
       [ 0., 10.,  0., 10.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  8.,  0.,  3.,  0.,  0.],
       [ 0.,  0.,  0.,  9.,  0.,  0.,  0.,  1.],
       [ 0.,  4.,  0.,  0.,  0.,  0.,  8.,  0.],
       [ 2.,  0.,  0.,  0.,  0.,  0.,  0.,  9.],
       [ 0.,  0.,  1.,  3.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 10.,  0., 10.],
       [ 0.,  0.,  0.,  0.,  4.,  2.,  0.,  0.]])

In [43]:
train + val + test

array([[ 0.,  0.,  0.,  1.,  7.,  2.,  3.,  8.],
       [ 9., 10.,  2.,  2.,  6.,  5.,  3.,  8.],
       [ 4.,  7.,  9.,  6.,  6., 10., 10.,  2.],
       [ 0.,  7.,  9.,  5.,  5., 10.,  9.,  1.],
       [ 7.,  6.,  3.,  8.,  3.,  4.,  3.,  2.],
       [ 0.,  0.,  9.,  9.,  6.,  8.,  9.,  0.],
       [ 3.,  5.,  4.,  4.,  3.,  3.,  9.,  0.],
       [10., 10., 10., 10.,  2.,  2.,  2.,  2.],
       [ 9.,  9.,  0.,  8.,  3.,  3.,  1.,  0.],
       [ 9.,  8., 10.,  9.,  3.,  4.,  2.,  1.],
       [ 4.,  4.,  3.,  3.,  9.,  9.,  8., 10.],
       [ 2.,  2.,  4.,  1.,  8., 10., 10.,  9.],
       [ 1.,  4.,  1.,  3.,  7., 10.,  7.,  8.],
       [ 3.,  3.,  2.,  1.,  1., 10.,  0., 10.],
       [ 9.,  9.,  8., 10.,  4.,  2.,  0.,  1.]])

In [44]:
(train + val + test) == ratings

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

In [45]:
fat = MatrixFactorization(dataframe = pd.DataFrame(train, columns = df.columns, index = df.index), 
                          K = 5, steps = 5000, alpha = 0.0001, beta = 0.2)

In [46]:
fat.fit()

Fatoração concluída. Tempo aproximado: 1 minuto(s).


In [47]:
pd.DataFrame(fat.predict(), columns = df.columns, index = df.index).round(2)

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,4.87,4.85,4.88,3.04,3.15,5.11,2.88,2.02
User_2,8.82,4.72,2.08,1.89,5.13,4.94,1.62,7.74
User_3,5.97,7.14,8.65,5.76,5.87,10.32,6.26,1.93
User_4,5.78,7.22,8.49,4.97,4.93,8.17,5.19,1.48
User_5,6.75,4.85,3.08,2.04,2.87,3.99,1.66,4.32
User_6,5.88,5.13,5.61,4.05,5.85,7.91,4.34,3.84
User_7,3.96,3.78,3.84,2.10,2.71,3.15,1.77,1.88
User_8,9.80,10.68,9.54,5.05,2.33,6.76,3.89,1.87
User_9,8.76,8.78,7.89,4.34,2.98,5.90,3.15,2.57
User_10,8.87,7.75,6.19,3.47,2.89,4.68,2.09,3.66


In [48]:
pd.DataFrame(val, columns = df.columns, index = df.index)

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,0.0,0.0,0.0,0.0,7.0,2.0,0.0,0.0
User_2,0.0,0.0,0.0,0.0,6.0,0.0,3.0,0.0
User_3,0.0,0.0,0.0,0.0,0.0,10.0,0.0,2.0
User_4,0.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0
User_5,0.0,0.0,0.0,8.0,0.0,0.0,0.0,2.0
User_6,0.0,0.0,0.0,0.0,0.0,8.0,9.0,0.0
User_7,0.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0
User_8,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0
User_9,9.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
User_10,0.0,0.0,10.0,0.0,0.0,4.0,0.0,0.0


In [52]:
class Similaridade():
    
    def __init__(self, dataframe):
        self.df = dataframe
        self.lista_usuarios = dataframe.index.tolist()
        self.lista_filmes = dataframe.columns.tolist()
        
    def similaridadeUsuarios(self, user, sorted_list = True):
        nota_user = self.df.loc[user, :].values 
        L = []
        for u in self.lista_usuarios:
            if u == user:
                continue
            else:
                nota_u = self.df.loc[u, :].values
                dist = euclid_dist(nota_user, nota_u)
                similaridade = 1 / (1 + dist)
                L.append([u,similaridade])
        if sorted_list:
            L.sort(key = lambda s: s[1], reverse = True)
            return L
        else:
            return L

    def similaridadeFilmes(self, filme, sorted_list = True): 
        nota_filme = self.df.T.loc[filme, :].values
        L = []
        for f in self.lista_filmes: 
            if f == filme:
                continue
            else:
                nota_f = self.df.T.loc[f, :].values
                dist = euclid_dist(nota_filme, nota_f)
                similaridade = 1 / (1 + dist)
                L.append([f,similaridade])
        if sorted_list:
            L.sort(key = lambda s: s[1], reverse = True)
            return L
        else:
            return L
        
    def similaridade_entre_dois_usuarios(self, user1, user2):
            return [x[1] for x in self.similaridadeUsuarios(user1) if x[0] == user2 ][0]

    def similaridade_entre_dois_filmes(self, filme1, filme2):
        return [x[1] for x in self.similaridadeFilmes(filme1) if x[0] == filme2 ][0]

In [54]:
df = getDF()

In [55]:
df

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0
User_3,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0
User_4,NaN,7.0,9.0,5.0,5.0,10.0,9.0,1.0
User_5,7.0,6.0,3.0,8.0,3.0,4.0,3.0,2.0
User_6,NaN,NaN,9.0,9.0,6.0,8.0,9.0,NaN
User_7,3.0,5.0,4.0,4.0,3.0,3.0,9.0,NaN
User_8,10.0,10.0,10.0,10.0,2.0,2.0,2.0,2.0
User_9,9.0,9.0,NaN,8.0,3.0,3.0,1.0,NaN
User_10,9.0,8.0,10.0,9.0,3.0,4.0,2.0,1.0


In [56]:
sim = Similaridade(dataframe = df)

In [57]:
sim.df

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0
User_3,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0
User_4,NaN,7.0,9.0,5.0,5.0,10.0,9.0,1.0
User_5,7.0,6.0,3.0,8.0,3.0,4.0,3.0,2.0
User_6,NaN,NaN,9.0,9.0,6.0,8.0,9.0,NaN
User_7,3.0,5.0,4.0,4.0,3.0,3.0,9.0,NaN
User_8,10.0,10.0,10.0,10.0,2.0,2.0,2.0,2.0
User_9,9.0,9.0,NaN,8.0,3.0,3.0,1.0,NaN
User_10,9.0,8.0,10.0,9.0,3.0,4.0,2.0,1.0


In [59]:
sim.lista_usuarios

['User_1',
 'User_2',
 'User_3',
 'User_4',
 'User_5',
 'User_6',
 'User_7',
 'User_8',
 'User_9',
 'User_10',
 'User_11',
 'User_12',
 'User_13',
 'User_14',
 'User_15']

In [60]:
sim.lista_filmes

['Filme_1',
 'Filme_2',
 'Filme_3',
 'Filme_4',
 'Filme_5',
 'Filme_6',
 'Filme_7',
 'Filme_8']

In [62]:
from sklearn.metrics.pairwise import nan_euclidean_distances

def euclid_dist(x, y):
    v = np.vstack([x, y])
    if np.isnan(nan_euclidean_distances(v)[0,1]):
        return 1e10
    else:
        return nan_euclidean_distances(v)[0,1]

In [63]:
sim.similaridadeUsuarios("User_1")

[['User_2', 0.1924840598000365],
 ['User_7', 0.08240267256634183],
 ['User_13', 0.07940855619903711],
 ['User_11', 0.07855272795675068],
 ['User_9', 0.0779292055122247],
 ['User_5', 0.07162563710668096],
 ['User_12', 0.06865934407787179],
 ['User_14', 0.06484157054036695],
 ['User_10', 0.06392879181901828],
 ['User_8', 0.06201113398134867],
 ['User_4', 0.057326814524408215],
 ['User_6', 0.05697049581409102],
 ['User_15', 0.056582425994704214],
 ['User_3', 0.05639139975154663]]

In [64]:
df.loc[['User_1', 'User_2']]

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_2,9.0,10.0,2.0,2.0,6.0,5.0,3.0,8.0


In [65]:
df.loc[['User_1', 'User_3']]

,Filme_1,Filme_2,Filme_3,Filme_4,Filme_5,Filme_6,Filme_7,Filme_8
User_1,NaN,NaN,NaN,1.0,7.0,2.0,3.0,8.0
User_3,4.0,7.0,9.0,6.0,6.0,10.0,10.0,2.0


In [66]:
sim.similaridadeFilmes("Filme_8")

[['Filme_5', 0.07147768141142521],
 ['Filme_6', 0.058133276275409755],
 ['Filme_7', 0.05648666578593548],
 ['Filme_2', 0.04067130131368077],
 ['Filme_1', 0.03899013204902304],
 ['Filme_3', 0.035830816042996094],
 ['Filme_4', 0.035779422208098904]]

In [67]:
df.T

,User_1,User_2,User_3,User_4,User_5,User_6,User_7,User_8,User_9,User_10,User_11,User_12,User_13,User_14,User_15
Filme_1,NaN,9.0,4.0,NaN,7.0,NaN,3.0,10.0,9.0,9.0,4.0,2.0,1.0,3.0,9.0
Filme_2,NaN,10.0,7.0,7.0,6.0,NaN,5.0,10.0,9.0,8.0,4.0,2.0,4.0,3.0,9.0
Filme_3,NaN,2.0,9.0,9.0,3.0,9.0,4.0,10.0,NaN,10.0,3.0,4.0,1.0,2.0,8.0
Filme_4,1.0,2.0,6.0,5.0,8.0,9.0,4.0,10.0,8.0,9.0,3.0,1.0,3.0,1.0,10.0
Filme_5,7.0,6.0,6.0,5.0,3.0,6.0,3.0,2.0,3.0,3.0,9.0,8.0,7.0,1.0,4.0
Filme_6,2.0,5.0,10.0,10.0,4.0,8.0,3.0,2.0,3.0,4.0,9.0,10.0,10.0,10.0,2.0
Filme_7,3.0,3.0,10.0,9.0,3.0,9.0,9.0,2.0,1.0,2.0,8.0,10.0,7.0,NaN,NaN
Filme_8,8.0,8.0,2.0,1.0,2.0,NaN,NaN,2.0,NaN,1.0,10.0,9.0,8.0,10.0,1.0
